<a href="https://colab.research.google.com/github/deborahpviana/Deep-Learning/blob/master/LeNet_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-gpu

In [0]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import matplotlib.pyplot as plt
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/Deep Learning")

!ls

In [0]:
########################################### PRE PROCESSAMENTO SEM DATA AUGMENTATION ##############################################
batch = 32
epochs = 10

img_x, img_y = 28, 28
input_format = (28, 28, 1)
num_classes = 10

(x_train, y_train), (x_val, y_val)  = mnist.load_data()

x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size = 0.5, random_state = 1)

x_train = x_train.reshape(x_train.shape[0], img_x, img_y, 1) 
x_val   = x_val.reshape(x_val.shape[0], img_x, img_y, 1)
x_test  = x_test.reshape(x_test.shape[0], img_x, img_y, 1)

x_train = x_train.astype('float32') / 255
x_val   = x_val.astype('float32') / 255
x_test  = x_test.astype('float32') / 255

y_train = to_categorical(y_train, num_classes)
y_val   = to_categorical(y_val, num_classes)
y_test  = to_categorical(y_test, num_classes)

In [0]:
##################################################### MODELO SEM DATA AUGMENTATION ###############################################
model = Sequential()

# C1 --> Primeira camada convolucional, com 6 filtros 5x5 com zeropadding
# Imput = 28x28  Output = 28x28
model.add(Conv2D(filters = 6, kernel_size=(5, 5), activation='relu', input_shape=input_format, padding='same'))

# M1 --> Primeira camada MaxPooling, com stride 2 sem zeropadding
# Imput = 28x28  Output = 14x14
model.add(MaxPooling2D(pool_size=(2, 2)))

# C2 --> Segunda camada convolucional, com 16 filtros 5x5 sem zeropadding
# Imput = 14x14  Output = 10x10
model.add(Conv2D(filters = 16, kernel_size=(5, 5), activation='relu'))

# M2 --> Segunda camada MaxPooling, com stride 2 sem zeropadding
# Imput = 10x10  Output = 5x5
model.add(MaxPooling2D(pool_size=(2, 2)))

# C4 --> Quarta camada convolucional, com 120 filtros 5x5 sem zeropadding
# Imput = 5x5  Output = 1x1
model.add(Conv2D(filters = 120, kernel_size=(5, 5), activation='relu'))

#
model.add(Flatten())

# F1 --> Primeira camada Full Conected, com 84 filtros
model.add(Dense(84, activation='relu'))

# Camada de Saída
model.add(Dense(num_classes, activation = 'softmax', kernel_initializer='glorot_normal'))

model.summary()

checkpoint = ModelCheckpoint('relu-model-{epoch:03d}-{val_acc:03f}-{val_loss:03f}.h5', 
                             verbose = 1, 
                             monitor = 'val_acc', 
                             save_best_only = True,
                             mode = 'max')

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics=['accuracy'])

historico = model.fit(x_train, y_train, batch_size = batch, validation_data=(x_val, y_val), verbose = 1, epochs = epochs, callbacks=[checkpoint])

In [0]:
########################################### PRE PROCESSAMENTO COM DATA AUGMENTATION ##############################################
batch = 32
epochs = 10

img_x, img_y = 28, 28
input_format = (img_x, img_y, 1)
num_classes = 10

(x_train, y_train), (x_val, y_val)  = mnist.load_data()

x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size = 0.5, random_state = 1)

x_train = x_train.reshape(x_train.shape[0], img_x, img_y, 1) 
x_val   = x_val.reshape(x_val.shape[0], img_x, img_y, 1)
x_test  = x_test.reshape(x_test.shape[0], img_x, img_y, 1)

y_train = to_categorical(y_train, num_classes)
y_val   = to_categorical(y_val, num_classes)
y_test  = to_categorical(y_test, num_classes)

x_test  = x_test.astype('float32') / 255

train_steps = len(x_train) / batch
val_steps = len(x_val) / batch

# DATA AUGMENTATION COMEÇA AQUI #
train_datagen = ImageDataGenerator(rescale = 1/255,
                                   rotation_range = 15,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='nearest')
                          
train_datagen.fit(x_train)

val_datagen = ImageDataGenerator(rescale = 1/255,
                                 rotation_range = 15,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 fill_mode='nearest')
val_datagen.fit(x_val)

train_generator = train_datagen.flow(x_train, y_train, batch_size = batch, shuffle = True)
val_generator = val_datagen.flow(x_val, y_val, batch_size = batch, shuffle = True)

In [0]:
##################################################### MODELO COM DATA AUGMENTATION ###############################################
model = Sequential()

# C1 --> Primeira camada convolucional, com 6 filtros 5x5 sem zeropadding
# Imput = 28x28  Output = 28x28
model.add(Conv2D(filters = 6, kernel_size=(5, 5), activation='relu', input_shape=input_format, padding='same'))

# M1 --> Primeira camada MaxPooling, com stride 2 sem zeropadding
# Imput = 28x28  Output = 14x14
model.add(MaxPooling2D(pool_size=(2, 2)))

# C2 --> Segunda camada convolucional, com 16 filtros 5x5 sem zeropadding
# Imput = 14x14  Output = 10x10
model.add(Conv2D(filters = 16, kernel_size=(5, 5), activation='relu'))

# M2 --> Segunda camada MaxPooling, com stride 2 sem zeropadding
# Imput = 10x10  Output = 5x5
model.add(MaxPooling2D(pool_size=(2, 2)))

# C4 --> Quarta camada convolucional, com 120 filtros 5x5 sem zeropadding
# Imput = 5x5  Output = 1x1
model.add(Conv2D(filters = 120, kernel_size=(5, 5), activation='relu'))

#
model.add(Flatten())

# F1 --> Primeira camada Full Conected, com 84 filtros
model.add(Dense(84, activation='tanh'))

# Camada de Saída
model.add(Dense(num_classes, activation = 'softmax', kernel_initializer='glorot_normal'))

model.summary()

checkpoint = ModelCheckpoint('relu-model-{epoch:03d}-{val_acc:03f}-{val_loss:03f}.h5', 
                             verbose = 1, 
                             monitor = 'val_acc', 
                             save_best_only = True,
                             mode = 'max')

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics=['accuracy'])

historico = model.fit_generator(generator = train_generator, 
                                steps_per_epoch = train_steps,
                                validation_data = val_generator,
                                validation_steps = val_steps,
                                epochs = epochs, 
                                verbose = 1, 
                                callbacks = [checkpoint])

In [0]:
# Fazendo os Teste
score = model.evaluate(x_test, y_test, verbose = 1)
print("Teste:\nAcurácia: {}\tLoss: {}".format(score[1], score[0]))

In [0]:
# Gráfico da Acurácia
plt.plot(historico.history['acc'], 'o--', color = '#FF1493', label = 'acc-train')    # Deep Pink
plt.plot(historico.history['val_acc'], 'o--', color = '#4169E1', label = 'acc-val')  # Royal Blue
plt.plot(score[1], 'o', color = '#006400', label = 'acc-test')                       # Dark Green
plt.title("Comparando os Valores de Acurácia")
plt.legend()
plt.grid(True)
plt.xlabel("Epochs")
plt.ylabel("Acurácia")
plt.show()

In [0]:
# Gráfico da Perda
plt.plot(historico.history['loss'], 'o--', color = '#DC143C', label = 'loss-train')    # Red - Crimson
plt.plot(historico.history['val_loss'], 'o--', color = '#FFD700', label = 'loss-val')  # Gold
plt.plot(score[0], 'o', color = '#A0522D', label = 'loss-test')                        # Brown
plt.title("Comparado os valores de Perda")
plt.legend()
plt.grid(True)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [0]:
# from keras.models import load_model

# model = load_model('')
# model.summary()

# score = model.evaluate(x_test, y_test, verbose = 1)
# print("Teste:\nAcurácia: {}\tLoss: {}".format(score[1], score[0]))